<a href="https://colab.research.google.com/github/mrhamedani/Gen-AI-projects-Pytorch/blob/main/9_TextGenerationLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # exercise 1
text="Hi, there!"
tokens=list(text)
print(tokens)

['H', 'i', ',', ' ', 't', 'h', 'e', 'r', 'e', '!']


In [ ]:
# exercise 2: word tokenization
text="It is unbelievably good!"
text=text.replace("!","")
tokens=text.split(" ")
print(tokens)

['It', 'is', 'unbelievably', 'good']


In [ ]:
# exercise 3 :
text="Hi, there!"
for x in list(",!"):
    text=text.replace(f"{x}",f" {x}")
tokens=text.split(" ")
print(tokens)

['Hi', ',', 'there', '!']


## Download & clean up the text

In [ ]:
!wget -O anna.txt "https://gitlab.com/amirbig44/pytorch-gan-course/-/raw/main/anna.txt?ref_type=heads&inline=false" # text of anna karenina

--2025-02-03 14:54:00--  https://gitlab.com/amirbig44/pytorch-gan-course/-/raw/main/anna.txt?ref_type=heads&inline=false
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2006029 (1.9M) [application/octet-stream]
Saving to: ‘anna.txt’

anna.txt            100%[===================>]   1.91M  --.-KB/s    in 0.08s   

2025-02-03 14:54:00 (24.8 MB/s) - ‘anna.txt’ saved [2006029/2006029]



In [ ]:
with open("./anna.txt","r") as f:
    text=f.read()
words=text.split(" ")
print(words[:20])

print(set(text.lower())) # unique characters in the text by set

['Chapter', '1\n\n\nHappy', 'families', 'are', 'all', 'alike;', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own\nway.\n\nEverything', 'was', 'in', 'confusion', 'in', 'the', "Oblonskys'"]
{'z', ' ', '0', 'l', '\n', '.', 'v', 'a', 's', 't', 'u', '?', 'f', "'", 'd', ';', ',', '-', 'c', '5', 'm', 'y', 'k', 'o', 'x', 'p', '9', 'q', '8', '2', '1', '_', 'e', 'h', 'g', 'n', ':', 'b', '(', '4', '!', '"', '3', '6', '`', 'j', 'w', '7', ')', 'r', 'i'}


In [ ]:
clean_text=text.lower().replace("\n", " ")
clean_text=clean_text.replace("-", " ")
for x in ",.:;?!$()/_&%*@'`":
    clean_text=clean_text.replace(f"{x}", f" {x} ")
clean_text=clean_text.replace('"', ' " ')
text=clean_text.split()

In [ ]:
from collections import Counter
word_counts = Counter(text)

words=sorted(word_counts, key=word_counts.get, reverse=True) # get unique words
print(words[:10])

[',', '.', 'the', '"', 'and', 'to', 'of', 'he', "'", 'a']


## Here we will make two dictionaries where the key values ​​are in the same place We will use it further

In [ ]:
text_length =len(text)
num_unique_words =len(words)
print(f"the text contains {text_length} words")
print(f"there are {num_unique_words} unique tokens")
word_to_int={v:k for k,v in enumerate(words)} # k is number of word repetition and ,v is word
int_to_word={k:v for k,v in enumerate(words)}
print({k:v for k,v in word_to_int.items() if k in words[:10]})
print({k:v for k,v in int_to_word.items() if v in words[:10]})

the text contains 437098 words
there are 12778 unique tokens
{',': 0, '.': 1, 'the': 2, '"': 3, 'and': 4, 'to': 5, 'of': 6, 'he': 7, "'": 8, 'a': 9}
{0: ',', 1: '.', 2: 'the', 3: '"', 4: 'and', 5: 'to', 6: 'of', 7: 'he', 8: "'", 9: 'a'}


In [ ]:
print(text[0:20])
wordidx=[word_to_int[w] for w in text]
print([word_to_int[w] for w in text[0:20]])  # The number of each word is the number of repetitions

['chapter', '1', 'happy', 'families', 'are', 'all', 'alike', ';', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own', 'way', '.', 'everything', 'was']
[208, 2755, 280, 2981, 83, 31, 2419, 35, 202, 685, 362, 38, 685, 10, 236, 147, 166, 1, 149, 12]


## Create batches of training data

In [ ]:
import torch

seq_len=100
xys=[]
for n in range(0, len(wordidx)-seq_len-1):  # 0 : 437098 -100 - 1
    x = wordidx[n:n+seq_len]                # x= 0:100   y = 1:101
    y = wordidx[n+1:n+seq_len+1]            # x= 1:101   y = 2:102
    xys.append((torch.tensor(x),(torch.tensor(y))))

In [ ]:
from torch.utils.data import DataLoader

torch.manual_seed(42)
batch_size=32
loader = DataLoader(xys, batch_size=batch_size, shuffle=True)

x,y=next(iter(loader))
print(x)
print(y)
print(x.shape,y.shape)

tensor([[  39,   31,    2,  ...,  688,  142,    7],
        [ 156, 5293,    0,  ...,   38,  330,    0],
        [   3,   97,    0,  ...,    0, 1774,   34],
        ...,
        [  16,  156,    9,  ...,  113,    5,  533],
        [   3,    4,   31,  ...,   98,    5,   98],
        [ 289,   19,   23,  ...,    9,  828,  550]])
tensor([[  31,    2, 2727,  ...,  142,    7,    0],
        [5293,    0,   16,  ...,  330,    0,    3],
        [  97,    0,    4,  ..., 1774,   34,    3],
        ...,
        [ 156,    9,  489,  ...,    5,  533,   27],
        [   4,   31,   25,  ...,    5,   98,    1],
        [  19,   23,    1,  ...,  828,  550,    1]])
torch.Size([32, 100]) torch.Size([32, 100])


# Build an LSTM model
### LSTM uses token and hidden state to create the next token and hidden state
The hidden state stores past information at each step, helping the LSTM understand dependencies between words in a sequence. In every step, a new hidden state is generated and passed to the next step. It works alongside the cell state to maintain both short-term and long-term information.

In [ ]:
import torch
from torch import nn
device="cuda" if torch.cuda.is_available() else "cpu"

class WordLSTM(nn.Module):
    def __init__(self, input_size=128, n_embed=128,
             n_layers=3, drop_prob=0.2):
        super().__init__()
        self.input_size=input_size
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_embed = n_embed
        vocab_size=len(word_to_int)
        self.embedding=nn.Embedding(vocab_size,n_embed)
        self.lstm = nn.LSTM(input_size=self.input_size,
            hidden_size=self.n_embed,
            num_layers=self.n_layers,
            dropout=self.drop_prob,batch_first=True)
        self.fc = nn.Linear(input_size, vocab_size) # vocab_size is the number of unique words
    def forward(self, x, hc):
        embed=self.embedding(x)
        x, hc = self.lstm(embed, hc) #hc is tuple of "hidden state" and "cell state"
        x = self.fc(x)
        return x, hc
    def init_hidden(self, n_seqs):             #Creates two Tensors (hidden state and cell state) with an initial value of zero.
        weight = next(self.parameters()).data       #In order that hc is not affected by previous epochs in each epoch, we set it to zero every time
        return (weight.new(self.n_layers,n_seqs, self.n_embed).zero_(), #3*32*128
                weight.new(self.n_layers,n_seqs, self.n_embed).zero_())

In [ ]:
model=WordLSTM().to(device)
print(model)

WordLSTM(
  (embedding): Embedding(12778, 128)
  (lstm): LSTM(128, 128, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=12778, bias=True)
)


In [ ]:
lr=0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

#Train the LSTM model
Because the train takes time, I use the train file

In [ ]:
# model.train()

# for epoch in range(50):
#     tloss=0
#     sh,sc = model.init_hidden(batch_size)
#     for i, (x,y) in enumerate(loader):
#         if x.shape[0]==batch_size:
#             inputs, targets = x.to(device), y.to(device)
#             optimizer.zero_grad()
#             output, (sh,sc) = model(inputs, (sh,sc))
#             loss = loss_func(output.transpose(1,2),targets)
#             sh,sc=sh.detach(),sc.detach()
#             loss.backward()
#             nn.utils.clip_grad_norm_(model.parameters(), 5)
#             optimizer.step()
#             tloss+=loss.item()
#         if (i+1)%1000==0:
#             print(f"at epoch {epoch} iteration {i+1}\
#             average loss = {tloss/(i+1)}")

#Generate text with the trained LSTM model
Generate text by predicting the next token.
in real we want Continue to generate prpmpt with 200 model tokens

In [ ]:
!wget -O wordLSTM.pth "https://gitlab.com/amirbig44/pytorch-gan-course/-/raw/main/wordLSTM.pth?ref_type=heads&inline=false"
# download the model weights file from the link

In [ ]:
model.load_state_dict(torch.load("wordLSTM.pth", map_location=device))

### For example, if our prompt is four words, we want the model to predict the next 196 words so that a total of 200 tokens are answered.We also put a condition that if the number of generated words is more than 100 (for example, to generate the 150th word), choose only the last 100.

In [ ]:
import numpy as np
def sample(model, prompt, length=200):  # length is the number of tokens to generate by the model
    model.eval()
    text = prompt.lower().split(' ') # text is a list of words prompt
    hc = model.init_hidden(1)      # 3*1*128
    length = length - len(text)
    for i in range(0, length):
        # if the text length is less than seq_len=100, use text to predict
        if len(text)<=seq_len:  #seq_len = 100
            x = torch.tensor([[word_to_int[w] for w in text]])
        # otherwise use the last seq_len=100 tokens to predict
        else:
            x = torch.tensor([[word_to_int[w] for w in text[-seq_len:]]])
        inputs = x.to(device)
        output, hc = model(inputs, hc)
        logits = output[0][-1]
        p = nn.functional.softmax(logits, dim=0).detach().cpu().numpy() # p is the probability normalized of each word
        idx = np.random.choice(len(logits), p=p)
        text.append(int_to_word[idx])
    text=" ".join(text)
    for m in ",.:;?!$()/_&%*@'`":
        text=text.replace(f" {m}", f"{m} ")
    text=text.replace('"  ', '"')
    text=text.replace("'  ", "'")
    text=text.replace('" ', '"')
    text=text.replace("' ", "'")
    return text

In [ ]:
import torch
torch.manual_seed(42)
np.random.seed(42)
print(sample(model, prompt='Anna and the prince'))


anna and the prince did not forget what he had not spoken.  when the softening barrier was not so long as he had talked to his brother,  all the hopelessness of the impression.  "official tail,  a man who had tried him,  though he had been able to get across his charge and locked close,  and the light round the snow was in the light of the altar villa.  the article in law levin was first more precious than it was to him so that if it was most easy as it would be as the same.  this was now perfectly interested.  when he had got up close out into the sledge,  but it was locked in the light window with their one grass,  and in the band of the leaves of his projects,  and all the same stupid woman,  and really,  and i swung his arms round that thinking of bed.  a little box with the two boys were with the point of a gleam of filling the boy,  noiselessly signed the bottom of his mouth,  and answering them took the red
